---
# Sieci neuronowe - projekt 3

---
## Załadowanie bibliotek

In [317]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from enum import Enum

---
## Załadowanie danych

In [318]:
temperature_data = pd.read_csv('temperature.csv')
wind_data = pd.read_csv('wind_speed.csv')

#wind_data = wind_data.fillna(wind_data.drop(['datetime'], axis=1).mean())

temperature_data = temperature_data.fillna(temperature_data.drop(['datetime'], axis=1).mean())
wind_data = wind_data.fillna(wind_data.drop(['datetime'], axis=1).mean())

temperature_data.head(5)
wind_data.head(5)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,2.432746,2.050738,2.786702,2.118116,1.219548,1.751458,2.463482,1.909063,2.763685,...,2.643212,3.210954,3.831897,3.380889,1.989339,2.722306,8.0,3.299406,3.024292,1.880047
1,2012-10-01 13:00:00,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,...,4.000000,7.000000,4.000000,3.000000,1.000000,0.000000,8.0,2.000000,2.000000,2.000000
2,2012-10-01 14:00:00,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,...,4.000000,7.000000,4.000000,3.000000,3.000000,0.000000,8.0,2.000000,2.000000,2.000000
3,2012-10-01 15:00:00,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,...,3.000000,7.000000,4.000000,3.000000,3.000000,0.000000,8.0,2.000000,2.000000,2.000000
4,2012-10-01 16:00:00,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,...,3.000000,7.000000,4.000000,3.000000,3.000000,0.000000,8.0,2.000000,2.000000,2.000000


---
## Przetworzenie danych

In [319]:

def calculate_means_by_days(data):
    data['datetime'] = pd.to_datetime(data['datetime'])
    data['date'] = data['datetime'].dt.date
    data = data.drop('datetime', axis=1)
    data = data.groupby('date').mean().reset_index()
    data = data[['date'] + [col for col in data.columns if col != 'date']]

    return data

def check_max_wind_speed_by_days(data):
    data['datetime'] = pd.to_datetime(data['datetime'])
    data['date'] = data['datetime'].dt.date
    data = data.drop('datetime', axis=1)
    data = data.groupby('date').max().reset_index()
    data = data[['date'] + [col for col in data.columns if col != 'date']]

    return data

temperature_data = calculate_means_by_days(temperature_data)
wind_data = check_max_wind_speed_by_days(wind_data)

In [283]:
temperature_data.head(5)

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01,284.620769,282.118197,289.416642,281.767262,291.846501,291.573495,293.358911,296.701739,285.476208,...,286.043165,288.569420,285.887980,287.371091,306.621486,304.248983,310.158846,304.400000,304.400000,303.500000
1,2012-10-02,286.145190,286.137728,292.958306,285.156888,295.890450,295.291472,297.248385,301.211968,289.771821,...,289.239595,290.892389,286.937931,289.013090,302.226773,302.787467,306.759071,303.900000,303.900000,302.675000
2,2012-10-03,285.528125,289.599792,296.929167,287.673958,299.008542,297.878750,300.691875,302.867083,291.205417,...,290.353542,290.065625,287.374583,289.020833,301.194375,301.687917,303.289583,301.561042,301.502500,301.258125
3,2012-10-04,284.373333,286.482500,295.687083,284.391667,295.997917,296.080833,301.820000,302.232917,293.096250,...,293.633750,291.987083,286.860833,290.043750,300.094167,299.940000,301.770208,299.139167,299.139167,298.924167
4,2012-10-05,283.757292,288.286042,290.635417,284.756250,292.948333,293.894375,300.628542,301.811250,292.829167,...,294.015833,294.043542,287.535208,289.517292,299.712083,300.153125,299.860000,298.877500,298.877500,297.547500


---
## Funkcje pomocnicze

In [303]:
def DrawData(X, y, plot_name):
    plt.style.use('dark_background')
    plt.figure(figsize=(10,5))
    axes = plt.gca()
    axes.set(xlabel="$X_1$", ylabel="$X_2$")
    plt.title(plot_name, fontsize=30)
    #plt.subplots_adjust(left=0.20)
    #plt.subplots_adjust(right=0.80)

    plt.scatter(X[0, :], X[1, :], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')

def DrawDataCompare(X, y, y_pred):
    plt.style.use('dark_background')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    ax1.set(xlabel="$X_1$", ylabel="$X_2$")
    ax1.set_title('Actual labels', fontsize=20)
    ax1.scatter(X[0, :], X[1, :], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')
    
    ax2.set(xlabel="$X_1$", ylabel="$X_2$")
    ax2.set_title('Predicted labels', fontsize=20)
    ax2.scatter(X[0, :], X[1, :], c=y_pred.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')
    
    plt.tight_layout()
    plt.show()

def GetClassificationData(name):
    file = pd.read_csv(name, sep=",")

    input = np.array(file[["x", "y"]])
    results = np.array(file["cls"] - 1)

    num_classes = results.max() + 1
    
    return input.T, results.T, num_classes

def get_data_for_city(data, part):
    n = len(data)
    ind = int(n*part)
    data_train = data[:ind].tolist()
    data_test = data[ind+1:].tolist()
    return data_train, data_test

def get_windows(data):
    n = len(data)
    windows_X = []
    windows_y = []

    for i in range(n - 4):
        window = data[i:i+3]
        windows_X.append(window)
        windows_y.append(data[i+4])

    return windows_X, windows_y

def convert_range(data, range_min, range_max):
    old_min = data.min()
    old_max = data.max()
    return (((data - old_min) * (range_max - range_min)) / (old_max - old_min)) + range_min


---
## Funkcje aktywacji

In [295]:
class Sigmoid():
    @staticmethod
    def calculate(x):
        x = np.clip( x, -500, 500 )
        return 1 / (1 + np.exp(-x))

    def calculateDeriv(self, x):
        return x * (1 - x)

    
class ReLU():
    @staticmethod
    def calculate(x):
        return np.maximum(x, 0)
    
    @staticmethod
    def calculateDeriv(x):
        return x > 0
    
    
class CrossEntropy():
    @staticmethod
    def calculate(y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return - (y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred)).mean()

    @staticmethod
    def calculateDeriv(y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return - (y / y_pred) + (1 - y) / (1 - y_pred)
    
class Softmax():
    @staticmethod
    def calculate(x):
        x = np.clip(x, 1e-15, 1 - 1e-15)
        e_x = np.exp(x - np.max(x, axis=0, keepdims=True))
        return e_x / np.sum(e_x, axis=0, keepdims=True)
    
    def calculateDeriv(self, x):
        value = self.calculate(x)
        return value * (1 - value)
    
class MSE:
    @staticmethod
    def calculate(y, y_pred):
        return ((y - y_pred) ** 2).mean()
    
    @staticmethod
    def calculateDeriv(y, y_pred):
        return -2*(y - y_pred) / y.shape[0]
    
class MAE:
    @staticmethod
    def calculate(y, y_pred):
        n = len(y)
        return np.sum(np.abs(y- y_pred)) / n
    
    @staticmethod
    def calculateDeriv(y, y_pred):
        n = len(y)
        return np.sign(y_pred - y) / n
        

class Tanh:
    @staticmethod
    def calculate(x):
        return np.tanh(x)

    @staticmethod
    def calculateDeriv(x):
        return 1 - x ** 2
    
class Linear:
    @staticmethod
    def calculate(x):
        return x

    @staticmethod
    def calculateDeriv(x):
        return np.ones_like(x)

---
## Sieć neuronowa

In [296]:
class NeuralNetworkStructure:
    def __init__(self, inputSize, outputSize, hiddenLayerSizes, hiddenLayerFunction, outputLayerFunction):
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenLayerSizes = hiddenLayerSizes
        self.layersSizes = hiddenLayerSizes + [outputSize]
        self.activationFunction = [hiddenLayerFunction] * len(hiddenLayerSizes) + [outputLayerFunction]
        self.layerInput = [None] * len(self.layersSizes)
        self.layerOutput = [None] * len(self.layersSizes)

        self.initializeWeights()

    def initializeWeights(self):
        self.weights = []
        self.bias = []
        
        previousLayerSize = self.inputSize
        for layerSize in self.layersSizes:
            self.weights.append(np.random.rand(layerSize, previousLayerSize) - 0.5)
            self.bias.append(np.random.rand(layerSize, 1) - 0.5)
            previousLayerSize = layerSize
            

class NeuralNetwork:
    def __init__(self, neuralNetworkStructure, epochs, learningRate, lossFunction, batchSize = 0):
        self.structure = neuralNetworkStructure
        self.lossFunction = lossFunction
        self.learningRate = learningRate
        self.epochs = epochs
        self.batchSize = batchSize

    def Forward(self, X):
        previous_layer = X #np.reshape(X, (X.shape[0], 1))
        for id in range(len(self.structure.layersSizes)):
            self.structure.layerInput[id] = self.structure.weights[id].dot(previous_layer) + self.structure.bias[id]
            self.structure.layerOutput[id] = self.structure.activationFunction[id].calculate(self.structure.layerInput[id])
            previous_layer = self.structure.layerOutput[id]
        return previous_layer
    
    
    def Backward(self, X, ExpectedY, PredictedY):
        if ExpectedY.ndim < 2:
            ExpectedY = np.reshape(ExpectedY, (1, -1))
        previous_layer_error = self.lossFunction.calculateDeriv(ExpectedY, PredictedY)
        
        for id in range(len(self.structure.layersSizes) -1, -1, -1):    
            previous_layer_output = self.structure.layerOutput[id - 1] if id != 0 else X
            
            delta = previous_layer_error * self.structure.activationFunction[id].calculateDeriv(self.structure.layerOutput[id])
            deltaW = np.dot(delta, previous_layer_output.T) 
            deltaB = np.sum(delta, axis=1, keepdims=True)
            previous_layer_error = np.dot(self.structure.weights[id].T, delta)
                                        
            self.structure.weights[id] -= self.learningRate * deltaW / ExpectedY.shape[1]
            self.structure.bias[id] -= self.learningRate * deltaB / ExpectedY.shape[1]
    
    def one_hot(self, Y):
        one_hot_Y = np.zeros((Y.max() + 1, Y.size))
        one_hot_Y[Y, np.arange(Y.size)] = 1
        return one_hot_Y
    
    def Train(self, X, ExpectedY):
   
        predictedY = self.Forward(X)
        self.Backward(X, ExpectedY, predictedY)
    
    def Test(self, train_inputs, train_results, test_inputs, schuffleParts = True):
        train_inputs = np.array(train_inputs).T
        train_results = np.array(train_results).T
        test_inputs = np.array(test_inputs).T
        
        if self.batchSize != 0:
            partsCount = (int)(np.shape(train_inputs)[1] / self.batchSize)
        else:
            partsCount = 1
        
        inputParts = np.array_split(train_inputs, partsCount, 1)
        if train_results.ndim < 2:
            resultsParts = np.array_split(train_results, partsCount, 0)
        else:
            resultsParts = np.array_split(train_results, partsCount, 1)
        
        for epoch in range(self.epochs):
            parts_range = list(range(partsCount))
            
            if schuffleParts == True:
                np.random.shuffle(parts_range)
        
            for i in parts_range:
                self.Train(inputParts[i], resultsParts[i])
            
        return self.Forward(test_inputs) 
    
    def Predict(self, test_inputs):
        predictedY = self.Forward(test_inputs)
        return predictedY


In [323]:
import numpy as np
import pandas as pd

cities = temperature_data.columns
cities = cities.drop(['date'])

for city in cities:

    np.random.seed(0)

    prev_min = temperature_data[city].min()
    prev_max = temperature_data[city].max()

    temperature_data_converted = convert_range(temperature_data[city], 0, 1)

    #windows_X, windows_y = get_windows(temperature_data_converted)

    #from sklearn.model_selection import train_test_split
    #train_windows_X, test_windows_X, train_windows_y, test_windows_y = train_test_split(windows_X, windows_y, test_size=0.7, random_state=0)

    data_train, data_test = get_data_for_city(temperature_data_converted, 0.3)

    train_windows_X, train_windows_y = get_windows(data_train)
    test_windows_X, test_windows_y = get_windows(data_test)

    nnS = NeuralNetworkStructure(
        inputSize = 3, 
        outputSize = 1, 
        hiddenLayerSizes = [64], 
        hiddenLayerFunction = Tanh(), 
        outputLayerFunction = Linear())

    nn = NeuralNetwork(  
        epochs = 3000, 
        learningRate = 0.01,
        neuralNetworkStructure = nnS,
        lossFunction = MAE(),
        batchSize=32)

    predictedPoints = nn.Test(train_windows_X, train_windows_y, test_windows_X)
    predictedPoints

    converted = convert_range(np.array(test_windows_y), prev_min, prev_max)
    predictedPoints = convert_range(np.array(predictedPoints), prev_min, prev_max)

    diff = converted - predictedPoints[0]
    err = abs(diff)

    predictedPoints
    print(f'City: {city}')
    print(f'Correct predictions [error <= 2]: {np.sum(err <= 2)}/{len(err)}, {np.sum(np.sum(err <=2))/len(err)*100}%')
    print(f'Max error: {np.max(err)}')
    print(f'Avg error: {np.mean(err)}')

City: Vancouver
Correct predictions [error <= 2]: 786/1316, 59.72644376899696%
Max error: 13.929207880483602
Avg error: 1.9783872783833218
City: Portland
Correct predictions [error <= 2]: 615/1316, 46.73252279635258%
Max error: 13.534938881385983
Avg error: 2.670467560606342
City: San Francisco
Correct predictions [error <= 2]: 727/1316, 55.243161094224924%
Max error: 11.658812799653617
Avg error: 2.1760050457091995
City: Seattle
Correct predictions [error <= 2]: 696/1316, 52.88753799392097%
Max error: 14.310711311415332
Avg error: 2.311143692203107
City: Los Angeles
Correct predictions [error <= 2]: 802/1316, 60.94224924012158%
Max error: 11.052762198917264
Avg error: 1.957470415738773
City: San Diego
Correct predictions [error <= 2]: 757/1316, 57.52279635258358%
Max error: 10.678584481526627
Avg error: 2.0555830150457153


KeyboardInterrupt: 

In [314]:
import numpy as np
import pandas as pd

cities = temperature_data.columns
cities = cities.drop(['date'])

for city in cities:

    np.random.seed(0)

    #windows_X, windows_y = get_windows(temperature_data[city])

    #from sklearn.model_selection import train_test_split
    #train_windows_X, test_windows_X, train_windows_y, test_windows_y = train_test_split(windows_X, windows_y, test_size=0.7, random_state=0)

    data_train, data_test = get_data_for_city(temperature_data[city], 0.3)

    train_windows_X, train_windows_y = get_windows(data_train)
    test_windows_X, test_windows_y = get_windows(data_test)

    nnS = NeuralNetworkStructure(
        inputSize = 3, 
        outputSize = 1, 
        hiddenLayerSizes = [64], 
        hiddenLayerFunction = Tanh(), 
        outputLayerFunction = Linear())

    nn = NeuralNetwork(  
        epochs = 3000, 
        learningRate = 0.01,
        neuralNetworkStructure = nnS,
        lossFunction = MAE(),
        batchSize=32)

    predictedPoints = nn.Test(train_windows_X, train_windows_y, test_windows_X)
    predictedPoints

    diff = test_windows_y - predictedPoints[0]
    err = abs(diff)

    predictedPoints
    print(f'City: {city}')
    print(f'Correct predictions [error <= 2]: {np.sum(err <= 2)}/{len(err)}, {np.sum(np.sum(err <=2))/len(err)*100}%')
    print(f'Max error: {np.max(err)}')
    print(f'Avg error: {np.mean(err)}')

City: Vancouver
Correct predictions [error <= 2]: 288/1316, 21.88449848024316%
Max error: 17.989957304115705
Avg error: 5.846982298594307
City: Portland
Correct predictions [error <= 2]: 291/1316, 22.112462006079028%
Max error: 19.314339554427193
Avg error: 6.049399896335404
City: San Francisco
Correct predictions [error <= 2]: 483/1316, 36.702127659574465%
Max error: 15.755855706365537
Avg error: 3.474601201013479
City: Seattle
Correct predictions [error <= 2]: 293/1316, 22.264437689969604%
Max error: 17.218885659196246
Avg error: 5.6349457349923044
City: Los Angeles
Correct predictions [error <= 2]: 345/1316, 26.21580547112462%
Max error: 15.840036284792632
Avg error: 4.469552232648598
City: San Diego
Correct predictions [error <= 2]: 400/1316, 30.3951367781155%
Max error: 16.101024799710103
Avg error: 3.89567112505848
City: Las Vegas
Correct predictions [error <= 2]: 176/1316, 13.37386018237082%
Max error: 23.375173581872332
Avg error: 8.993039022463861
City: Phoenix
Correct predict

KeyboardInterrupt: 

In [327]:
import numpy as np
import pandas as pd


def convert(x):
  return 1 if x >= 6 else 0

cities = wind_data.columns
cities = cities.drop(['date'])

for city in cities:
    
    np.random.seed(0)

    wind_data_class = np.apply_along_axis(np.vectorize(convert), -1, wind_data[city])

    windows_X, windows_y = get_windows(wind_data_class)

    from sklearn.model_selection import train_test_split
    train_windows_X, test_windows_X, train_windows_y, test_windows_y = train_test_split(windows_X, windows_y, test_size=0.7, random_state=0)

    def one_hot(Y):
        one_hot_Y = np.zeros((Y.max() + 1, Y.size))
        one_hot_Y[Y, np.arange(Y.size)] = 1
        return one_hot_Y

    train_windows_y = one_hot(np.array(train_windows_y).astype(int)).T
    train_windows_y

    functions = [Sigmoid(), Tanh(), ReLU()]
    
    for function in functions:
        
        nnS = NeuralNetworkStructure(
            inputSize = 3, 
            outputSize = 2, 
            hiddenLayerSizes = [64], 
            hiddenLayerFunction = function, 
            outputLayerFunction = Softmax())

        nn = NeuralNetwork(  
            epochs = 1000, 
            learningRate = 0.01,
            neuralNetworkStructure = nnS,
            lossFunction = CrossEntropy(),
            batchSize=32)

        predictedPoints = nn.Test(train_windows_X, train_windows_y, test_windows_X)
        diff = np.argmax(predictedPoints, 0) == test_windows_y
        print(f'{city}: {type(function)} Correct predictions: {np.sum(diff)}/{len(diff)}, {np.sum(diff)/len(diff)*100}%')

Vancouver: <class '__main__.Sigmoid'> Correct predictions: 806/1319, 61.10689916603488%
Vancouver: <class '__main__.Tanh'> Correct predictions: 813/1319, 61.63760424564063%
Vancouver: <class '__main__.ReLU'> Correct predictions: 813/1319, 61.63760424564063%
Portland: <class '__main__.Sigmoid'> Correct predictions: 942/1319, 71.4177407126611%
Portland: <class '__main__.Tanh'> Correct predictions: 915/1319, 69.3707354056103%
Portland: <class '__main__.ReLU'> Correct predictions: 942/1319, 71.4177407126611%
San Francisco: <class '__main__.Sigmoid'> Correct predictions: 1015/1319, 76.95223654283548%
San Francisco: <class '__main__.Tanh'> Correct predictions: 995/1319, 75.43593631539045%
San Francisco: <class '__main__.ReLU'> Correct predictions: 1004/1319, 76.11827141774071%
Seattle: <class '__main__.Sigmoid'> Correct predictions: 972/1319, 73.69219105382867%
Seattle: <class '__main__.Tanh'> Correct predictions: 959/1319, 72.70659590598939%
Seattle: <class '__main__.ReLU'> Correct predicti

KeyboardInterrupt: 

In [315]:
import numpy as np
import pandas as pd

cities = wind_data.columns
cities = cities.drop(['date'])

for city in cities:

    np.random.seed(0)

    prev_min = wind_data[city].min()
    prev_max = wind_data[city].max()

    wind_data_regression = convert_range(wind_data[city], 0, 1)

    windows_X, windows_y = get_windows(wind_data_regression)

    from sklearn.model_selection import train_test_split
    train_windows_X, test_windows_X, train_windows_y, test_windows_y = train_test_split(windows_X, windows_y, test_size=0.7, random_state=0)

    nnS = NeuralNetworkStructure(
        inputSize = 3, 
        outputSize = 1, 
        hiddenLayerSizes = [64], 
        hiddenLayerFunction = Tanh(), 
        outputLayerFunction = Linear())

    nn = NeuralNetwork(  
        epochs = 3000, 
        learningRate = 0.01,
        neuralNetworkStructure = nnS,
        lossFunction = MAE(),
        batchSize=32)

    predictedPoints = nn.Test(train_windows_X, train_windows_y, test_windows_X)
    predictedPoints

    converted = convert_range(np.array(test_windows_y), prev_min, prev_max)
    predictedPoints = convert_range(np.array(predictedPoints), prev_min, prev_max)

    diff = (converted >= 6) == (predictedPoints[0] >= 6) 

    print(f'{city} Correct predictions: {np.sum(diff)}/{len(diff)}, {np.sum(diff)/len(diff)*100}%')

Vancouver Correct predictions: 662/1319, 50.18953752843063%
Portland Correct predictions: 686/1319, 52.00909780136467%
San Francisco Correct predictions: 1082/1319, 82.03184230477635%
Seattle Correct predictions: 485/1319, 36.770280515542076%
Los Angeles Correct predictions: 809/1319, 61.33434420015163%
San Diego Correct predictions: 1064/1319, 80.66717210007582%
Las Vegas Correct predictions: 848/1319, 64.29112964366944%


KeyboardInterrupt: 